<a href="https://colab.research.google.com/github/SauloHenriqueAguiar/OptimizeLLMs/blob/main/zeroshotimageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel
from io import BytesIO
import json


In [8]:
url = "https://upload.wikimedia.org/wikipedia/commons/5/58/MountainLion.jpg"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    image = Image.open(BytesIO(response.content)).convert("RGB")
    image.show()  # só para visualização no Colab
else:
    raise Exception(f"Erro ao baixar imagem: {response.status_code}")

In [9]:
# Frases em português para classificação
texts = ["um cachorro", "um gato", "um leão", "um puma", "uma casa", "um carro", "uma floresta"]


In [10]:
# Carregar modelo CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [11]:
# Preparar entrada
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)


In [12]:
# Passar pelo modelo
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # similaridade imagem-texto
    probs = logits_per_image.softmax(dim=1)  # converter para probabilidades


In [13]:
# Mostrar resultado
for label, prob in zip(texts, probs[0]):
    print(f"{label}: {prob.item()*100:.2f}%")

um cachorro: 1.51%
um gato: 5.46%
um leão: 0.31%
um puma: 89.69%
uma casa: 2.34%
um carro: 0.54%
uma floresta: 0.16%


In [14]:
# Melhor palpite
best_idx = probs.argmax().item()
print("\n🔍 Melhor descrição segundo o modelo:", texts[best_idx])


🔍 Melhor descrição segundo o modelo: um puma
